In [2]:
import numpy as np 
import pandas as pd 
import os
from PIL import Image
import matplotlib.pyplot as plt
from transformers import BeitFeatureExtractor, BeitModel
import torch
from transformers import BertTokenizer, BertModel

In [7]:
folder_name = 'petfinder-adoption-prediction' # 'petfinder-adoption-prediction/train'
train = f'{folder_name}/train/train.csv'
test = f'{folder_name}/test/test.csv'
main_data = pd.read_csv(train)
test_data = pd.read_csv(test)

In [9]:
main_data

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,NaN,2,266,0,3,1,0,0,2,...,1,4,0,41326,61c84bd7bcb6fb31d2d480b1bcf9682e,0,I have 4 kittens that need to be adopt urgentl...,dc0935a84,3.0,2
14989,2,Serato & Eddie,60,265,264,3,1,4,7,2,...,1,2,0,41326,1d5096c4a5e159a3b750c5cfcf6ceabf,0,Serato(female cat- 3 color) is 4 years old and...,a01ab5b30,3.0,4
14990,2,Monkies,2,265,266,3,5,6,7,3,...,1,5,30,41326,6f40a7acfad5cc0bb3e44591ea446c05,0,"Mix breed, good temperament kittens. Love huma...",d981b6395,5.0,3
14991,2,Ms Daym,9,266,0,2,4,7,0,1,...,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4


In [8]:
test_data

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt
0,2,Dopey & Grey,8,266,266,1,2,6,7,1,...,2,1,2,0,41326,2ece3b2573dcdcebd774e635dca15fd9,0,"Dopey Age: 8mths old Male One half of a pair, ...",e2dfc2935,2.0
1,2,Chi Chi,36,285,264,2,1,4,7,2,...,1,2,1,0,41326,2ece3b2573dcdcebd774e635dca15fd9,0,"Please note that Chichi has been neutered, the...",f153b465f,1.0
2,2,Sticky,2,265,0,1,6,7,0,2,...,2,1,1,200,41326,e59c106e9912fa30c898976278c2e834,0,"Sticky, named such because of his tendency to ...",3c90f3f54,4.0
3,1,Dannie & Kass [In Penang],12,307,0,2,2,5,0,2,...,1,1,2,0,41326,e59c106e9912fa30c898976278c2e834,0,Dannie and Kass are mother and daughter. We en...,e02abc8a3,5.0
4,2,Cuddles,12,265,0,1,2,3,7,2,...,1,1,1,0,41326,e59c106e9912fa30c898976278c2e834,0,"Extremely cuddly cat, hence the origin of his ...",09f0df7d1,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,1,Hugo,5,307,307,1,1,2,0,2,...,2,1,2,150,41401,18ec8ca4486bc2760de0bd5390cee30c,0,Found on the streets. Treated for mange. They ...,ae57f8d52,5.0
3968,1,Spot,30,307,307,1,1,2,7,2,...,2,1,1,0,41326,d83be5f5e2d04e24d7376e99eafd8708,0,Very good guard dog. Healthy was found in Fron...,83432904d,2.0
3969,2,NaN,5,300,0,3,1,2,4,2,...,2,1,6,0,41401,30aa45fdbe45e39d5614ef583b569073,0,these cat's mother was killed when they was ne...,399013029,1.0
3970,1,Smokey,24,307,0,2,5,7,0,2,...,2,1,1,0,41325,087903c2819a6297519c93d962b488d5,0,"smokey is good family pet. very obedient,so lo...",fd80b8c80,3.0


# logistic regression

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [44]:
X = main_data.drop(columns=['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID'])  # Features
y = main_data['AdoptionSpeed']  # Labels
# X = pd.get_dummies(X, columns=['RescuerID'])
# categorical_columns = []
# for column in X.columns:
#     if X[column].dtype == 'object':
#         categorical_columns.append(column)
# # Print the list of categorical columns
# print("Categorical Columns:", categorical_columns) # Categorical Columns: ['Name', 'RescuerID', 'Description', 'PetID']

# Split the data into training and testing sets (adjust test_size as needed)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a logistic regression model
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', verbose=0)
model.fit(X, y)

# Predict the labels on the test set
y_pred = model.predict(X)

print(accuracy_score(y, y_pred))
# print(classification_report(y_test, y_pred, zero_division=1))
# classification_rep = classification_report(y_test, y_pred, zero_division=1)

0.32215033682385114


In [45]:
testX = test_data.drop(columns=['Name', 'Description', 'PetID', 'RescuerID'])  # Features
predicted_labels = model.predict(testX)
test_result = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': predicted_labels})
print(test_result)
test_result.to_csv('Results/BL_lr1.csv', index=False)

          PetID  AdoptionSpeed
0     e2dfc2935              2
1     f153b465f              4
2     3c90f3f54              2
3     e02abc8a3              4
4     09f0df7d1              4
...         ...            ...
3967  ae57f8d52              4
3968  83432904d              4
3969  399013029              4
3970  fd80b8c80              4
3971  493ed84ae              2

[3972 rows x 2 columns]


In [55]:
X = main_data.drop(columns=['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID'])  # Features
y = main_data['AdoptionSpeed']  # Labels
X = pd.get_dummies(X, columns=['Type', 'Gender'])
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', verbose=0)
model.fit(X, y)
y_pred = model.predict(X)
print(accuracy_score(y, y_pred))

0.3206162876008804


In [56]:
testX = test_data.drop(columns=['Name', 'Description', 'PetID', 'RescuerID'])  # Features
testX = pd.get_dummies(testX, columns=['Type', 'Gender'])
predicted_labels = model.predict(testX)
test_result = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': predicted_labels})
print(test_result)
test_result.to_csv('Results/BL_lr2.csv', index=False)

          PetID  AdoptionSpeed
0     e2dfc2935              2
1     f153b465f              4
2     3c90f3f54              2
3     e02abc8a3              4
4     09f0df7d1              4
...         ...            ...
3967  ae57f8d52              4
3968  83432904d              4
3969  399013029              4
3970  fd80b8c80              4
3971  493ed84ae              2

[3972 rows x 2 columns]


# cart

In [53]:
from sklearn.tree import DecisionTreeClassifier

In [57]:
X = main_data.drop(columns=['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID'])  # Features
y = main_data['AdoptionSpeed']  # Labels

model = DecisionTreeClassifier()
model.fit(X, y)

# Predict the labels on the test set
y_pred = model.predict(X)

print(accuracy_score(y, y_pred))
# print(classification_report(y_test, y_pred, zero_division=1))
# classification_rep = classification_report(y_test, y_pred, zero_division=1)

0.9823917828319882


In [58]:
testX = test_data.drop(columns=['Name', 'Description', 'PetID', 'RescuerID'])  # Features
predicted_labels = model.predict(testX)
test_result = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': predicted_labels})
print(test_result)
test_result.to_csv('Results/BL_cart1.csv', index=False)

          PetID  AdoptionSpeed
0     e2dfc2935              2
1     f153b465f              1
2     3c90f3f54              1
3     e02abc8a3              1
4     09f0df7d1              4
...         ...            ...
3967  ae57f8d52              2
3968  83432904d              4
3969  399013029              2
3970  fd80b8c80              4
3971  493ed84ae              1

[3972 rows x 2 columns]


# CatBoost

In [60]:
import catboost
from catboost import CatBoostClassifier

In [64]:
X = main_data.drop(columns=['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID'])  # Features
y = main_data['AdoptionSpeed']  # Labels

non_numeric_columns = ['Gender', 'Breed1', 'Breed2', 'Color1', 'Color2', 'Color3']
for col in non_numeric_columns:
    X[col] = X[col].astype('category')

cat_naive = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=10,
    loss_function='MultiClass'
)
cat_naive.fit(X, y, cat_features=non_numeric_columns, verbose=0)

In [65]:
# Predict the labels on the test set
y_pred = cat_naive.predict(X)
print(accuracy_score(y, y_pred))

0.6982591876208898


In [74]:
testX = test_data.drop(columns=['Name', 'Description', 'PetID', 'RescuerID'])  # Features
predicted_labels = cat_naive.predict(testX)
test_result = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': [x[0] for x in predicted_labels]})
print(test_result)
test_result.to_csv('Results/BL_cat1.csv', index=False)

          PetID  AdoptionSpeed
0     e2dfc2935              4
1     f153b465f              1
2     3c90f3f54              2
3     e02abc8a3              4
4     09f0df7d1              4
...         ...            ...
3967  ae57f8d52              4
3968  83432904d              4
3969  399013029              4
3970  fd80b8c80              4
3971  493ed84ae              4

[3972 rows x 2 columns]


# Random Forest

In [76]:
from sklearn.ensemble import RandomForestClassifier

In [77]:
X = main_data.drop(columns=['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID'])  # Features
y = main_data['AdoptionSpeed']  # Labels

rf_naive = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    min_samples_split=10, 
    min_samples_leaf=5, 
    max_features='sqrt', 
    n_jobs=-1, 
    random_state=42,
    verbose=0
)
rf_naive.fit(X, y)

# Predict the labels on the test set
y_pred = model.predict(X)

print(accuracy_score(y, y_pred))
# print(classification_report(y_test, y_pred, zero_division=1))
# classification_rep = classification_report(y_test, y_pred, zero_division=1)

0.9823917828319882


In [78]:
testX = test_data.drop(columns=['Name', 'Description', 'PetID', 'RescuerID'])  # Features
predicted_labels = rf_naive.predict(testX)
test_result = pd.DataFrame({'PetID': test_data['PetID'], 'AdoptionSpeed': predicted_labels})
print(test_result)
test_result.to_csv('Results/BL_rf1.csv', index=False)

          PetID  AdoptionSpeed
0     e2dfc2935              4
1     f153b465f              4
2     3c90f3f54              2
3     e02abc8a3              4
4     09f0df7d1              4
...         ...            ...
3967  ae57f8d52              4
3968  83432904d              4
3969  399013029              4
3970  fd80b8c80              4
3971  493ed84ae              4

[3972 rows x 2 columns]
